In [ ]:
import pandas as pd

from frenchtext.core import *
from frenchtext.datasets import *
from frenchtext.chars import *

In [ ]:
config.datasets

WindowsPath('C:/Users/Laurent.000/.frenchtext/datasets')

In [ ]:
config["datasets_path"] = "D:/Laurent/.frenchtext/datasets"
config.datasets.mkdir(parents=True, exist_ok=True)

In [ ]:
download_all_datasets()

In [ ]:
dfcharsnorm = pd.read_csv(chardatadir / "charset-fr.csv", sep=";")
delimiters = dfcharsnorm[(dfcharsnorm["SubCategory"]=="space") | (dfcharsnorm["Category"]=="emoticon")]["Char"].values
delimiters

array([' ', '\n', '\t', '🙂', '😉', '😀', '😂', '😁', '😊', '🙁', '😅', '😍', '😃',
       '😡', '🤣', '😄', '🤔', '😎', '😭', '👹', '😱', '😜', '😋', '\U0001f929',
       '🙄', '😆', '😛', '\U0001f92a', '😢', '😇', '🤦', '💪', '👉', '👍', '👏',
       '🙏', '🙌', '👇', '👊', '👎', '👌', '✌', '✊', '⚠', '🔴', '🔥', '🏆', '⚽',
       '💡', '🚨', '💥', '⚡', '♫', '♂', '♀', '🎉', '✍', '✉', '✝'],
      dtype=object)

In [ ]:
import re
regexPattern = '|'.join(map(re.escape, delimiters))

In [ ]:
from fastprogress.fastprogress import progress_bar

indexes = []
positions = []
tokens = []

norm = TextNormalizer()

for datasetfile in list_dataset_files()[3:]:
    datasetdf = read_dataset_file(datasetfile)
    pbar = progress_bar(range(len(datasetdf)), leave=False, parent=None)
    pbar.comment = datasetfile
    pbar.update(0)
    lines = 0
    for idx,row in datasetdf.iterrows():
        if row["Lang"] == "fr" or row["Lang"] == "?":
            normtext = norm(row["Text"])
            for pos,token in enumerate(re.split(regexPattern, normtext.output)):
                indexes.append(idx)
                positions.append(pos)
                tokens.append(token)
                lines+=1
        if idx % 100 == 0:
            pbar.update(idx)
        if lines > 100000000:
            pbar.update(idx)
            break
    pbar.update(idx)
    tokensdf = pd.DataFrame.from_dict({"Index":indexes, "Position":positions,  "Token":tokens})
    tokensdf = tokensdf.astype({'Index': 'int32', 'Position' : 'int16'}, copy=False)
    tokensdf.to_feather(config.datasets / (datasetfile + ".tokens1.feather"))

Loaded dataframe for dataset comparateur : 735034 text blocks


Loaded dataframe for dataset crédit : 59741 text blocks


Loaded dataframe for dataset forum : 5433843 text blocks


MemoryError: 

In [ ]:
tokensdf = pd.read_feather(config.datasets / ("assurance.tokens1.feather"))

In [ ]:
%time tokensdf[tokensdf["Token"].str.contains(',')].head(50)

Wall time: 6.01 s


,Index,Position,Token
27,2,14,"Frileuse,"
47,6,4,"personnes,"
50,6,7,"protégeons,"
67,6,24,"prévoyance,"
68,6,25,"retraite,"
96,6,53,"mieux,"
116,7,3,"manière,"
124,7,11,"mécanique,"
140,7,27,"effet,"
163,9,6,"classique,"


In [ ]:
https://setuptools.readthedocs.io/en/latest/setuptools.html#including-data-files